In [0]:
import $file.hw4stdlib
import hw4stdlib._

Cannot resolve $file import: /home/jovyan/work/hw4stdlib.sc

: 

# Homework 4

In this assignment we will develop some helpful structures for building the interpreter.Then we will write the first full interpreter for Lettuce. Remember that lettuce is a functional language with let bindings. It is very similiar to a language called ML. Here we want to implement the interpreter for the language from the ground up. We will use most of the existing standard library we have developed while we write this, especially the equality functions and Dictionary data type from earlier in the homework.


## Dictionaries

We will need to define the datatype $\texttt{Dictionary}$ before we write the first interpreter for lettuce. We'll begin by giving the definition below:

$$
\begin{aligned}
\textbf{Dictionary}\ A\ B\ ::&= \text{EmptyDict} \\
&\mid\ \text{Entry}\ A\ B\ (\textbf{Dictionary}\ A\ B)
\end{aligned}
$$

This will create dictionaries of key-value pairs which will be very useful for implementing contexts in the next homework assignment. The definition of this type in Scala is given below:

In [ ]:
// YOUR CODE HERE
???

For both of the following functions you will need to provide a parameter for an equality-checking function. This is so we can check if keys are equal to eachother

### Membership(In) (5pts)

Write a function called `isIn` which returns `True` if the Dictionary contains a given key and `False` if it does not.

In [ ]:
// YOUR CODE HERE
???

In [ ]:
assert(isIn(nat_eq, Entry(five, True, EmptyDict), five) == True)
assert(isIn(nat_eq, Entry(eight, True, Entry(five, True, EmptyDict)), five) == True)
assert(isIn(nat_eq, Entry(five, True, EmptyDict), six) == False)
assert(isIn(nat_eq, EmptyDict, seven) == False)
passed(5)

### Lookup (5 pts)

Write a function that retrieves a value that corresponds to a key in a dictionary. If the key is not in the map then return Nothing.

$$
\texttt{lookup} : (a \rightarrow a \rightarrow \mathbb{B}) \rightarrow \texttt{Dict}\ a\ b \rightarrow a \rightarrow \texttt{Maybe}\ b
$$

In [ ]:
// YOUR CODE HERE
???

In [ ]:
assert(lookup(nat_eq, Entry(five, "ham", EmptyDict), five) == Just("ham"))
assert(lookup(nat_eq, Entry(eight, seven, Entry(five, three, EmptyDict)), five) == Just(three))
assert(lookup(nat_eq, Entry(five, True, EmptyDict), six) == None)
assert(lookup(nat_eq, EmptyDict, seven) == None)
passed(5)

### Practice using these functions (5 pts)
#### Part A
Create the following dictionary as a scala value using this definition:
```
{
    five: True,
    six: False,
    seven: False
}
```
Place it in a variable named "`ans_3a`"

In [ ]:
// Solution should look like `val ans3 = ???`
// YOUR CODE HERE
???

In [ ]:
assert(lookup(nat_eq, ans_3a, five) == Just(True))
assert(lookup(nat_eq, ans_3a, six) == Just(False))
assert(lookup(nat_eq, ans_3a, seven) == Just(False))
passed(2)

#### Part B
For the following, use the super secret `secret_dict` variable defined below. Use the functions you just defined to complete the following:

1. Look up the value mapped by key `Positive(one)` and place it in variable `ans_3b1` 
2. Look up the value mapped by key `Positive(three)` and place it in variable `ans_3b2`
3. Check whether the key `Negative(three)` is in the map and place True or False in variable `ans_3b3`
4. Check whether the key `Negative(one)` is in the map and place True or False in variable `ans_3b4`


**Note**: For 1 and 2 remember to extract it from the maybe type, set the variable to the string `"does not exist"` if you get `None` instead of `Just(x)`.

In [ ]:
val secret_dict = Entry(Positive(three), "abc",
                        Entry(Negative(one), "xyz",
                              Entry(Positive(one), "MIB", EmptyDict)))

In [ ]:
// YOUR CODE HERE
???

In [ ]:
// Hidden tests (3 pts)

# The Lettuce Interpreter



## Problem 1 - The Values (5 points)

Recall that we define interpreters as functions that take in expressions in the form of abstract syntax and give a value as output. In symbols:

$$
eval : Expr \rightarrow Value
$$

It follows that we will need to create a datatype to represent the values that may be computed by lettuce. Bellow is the grammar that defines lettuce values. Implemenet this as a `sealead trait` in Scala below:

$$
\begin{align}
\textbf{Value} ::=&\ NumVal\ \mathbb{Z}\\
\mid&\ BinVal\ \mathbb{B}\\
\mid&\ Error
\end{align}
$$

In [3]:
sealed trait Value
case class NumVal(f: Int) extends Value
case class BinValue(b: String) extends Value
case class Error() extends Value

defined trait Value
defined class NumVal
defined class BinValue
defined class Error

If your definition was correct then the values in the cell below should compile.

In [3]:
val v1 = NumVal(Positive(Succ(Succ(Zero))))
val v2 = BinVal(False)
val v3 = Error
passed(5)

cmd3.sc:1: not found: value Positive
val v1 = NumVal(Positive(Succ(Succ(Zero))))
                ^cmd3.sc:1: not found: value Succ
val v1 = NumVal(Positive(Succ(Succ(Zero))))
                         ^cmd3.sc:1: not found: value Succ
val v1 = NumVal(Positive(Succ(Succ(Zero))))
                              ^cmd3.sc:1: not found: value Zero
val v1 = NumVal(Positive(Succ(Succ(Zero))))
                                   ^cmd3.sc:2: not found: value BinVal
val v2 = BinVal(False)
         ^cmd3.sc:2: not found: value False
val v2 = BinVal(False)
                ^cmd3.sc:4: not found: value passed
val res3_3 = passed(5)
             ^Compilation Failed

: 

## Problem 2 - The Expressions (10 points)

There are many possible expressions in the Lettuce language. We also use an abstract syntax to represent this. Here is the grammar to remind you of all the syntactic elements in Lettuce:

$$\begin{array}{rcll}
\mathbf{Expr} & ::= & Const(\mathbb{Z}) \\
 & | & Bin(\mathbb{B}) \\
 & | & Ident(\mathbf{String}) \\
 & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Minus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Mult (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Pow (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Neg (\mathbf{Expr}) \\
 & | & Eq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & And ( \mathbf{Expr}, \mathbf{Expr} ) \\
 & | & Or ( \mathbf{Expr}, \mathbf{Expr} ) \\
 & | & IfThenElse(\mathbf{Expr}, \mathbf{Expr}, \mathbf{Expr}) \\
 & | & Let( \mathbf{String}, \mathbf{Expr}, \mathbf{Expr}) \\
\end{array}$$

Now define a `sealed trait` for expressions in Lettuce:

In [ ]:
// YOUR CODE HERE
???

If you defined `Expr` properly then the following expressions should compile:

In [ ]:
val ex1 = Const(Positive(five))
val ex2 = Const(Negative(three))
val ex3 = Bin(True)
val ex4 = Ident("x")
passed(5)

In [ ]:
val ex5 = Plus(ex1, ex2)
val ex6 = Let("x", ex5, Mult(ex4, ex4))
passed(5)

## Problem 3 - The Interpreter (30 pts)

We now need to define the interpreter for this language. It is a function of the form:

$$
eval : Expr \rightarrow Value
$$

Except now that we have an execution environment($\sigma$) we need some way to include sigma in this function. This will be as an additional argument:

$$
eval : \sigma \rightarrow Expr \rightarrow Value
$$

We will let $\sigma$ be a `Dictionary` as we defined in last week's homework. To add something to a dicitonary use the `Entry` constructor to add on a new element. This should be very similiar to the way `Cons` works on lists.

Below we have given you the skeleton of the function `eval`. Fill in each case for the interpreter so that it can interpret all possible Lettuce expressions. The inference rules we covered in the classroom should be very helpful with this.

Hint: It may be helpful to define some helper functions for the inference rules that have similar forms. These are the rules like binary operations for arithmetic, bool, etc. Just like we lumped some of these rules together into a single rule, we can do the same for our interpreter in the form of auxillary functions. The bonus `eval_bin` function from Homework 4 may be a good place for inspiration.

Hint: Be ready to write some nested case matches. Most of your cases should have two nestings. One for the Sytnax element and another for evaluating the expressions it may contain. This is not true for all expressions but most will 

In [ ]:
// YOUR CODE HERE
???

If this was defined correctly then the definitions above should evaluate:

In [ ]:
val ex1 = Const(Positive(five))
val ex2 = Const(Negative(three))
val ex3 = Bin(True)
val ex4 = Ident("x")
val ex5 = Plus(ex1, ex2)
val ex6 = Let("x", ex5, Mult(ex4, ex4))

In [ ]:
assert(eval(EmptyDict, ex3) == BinVal(True))
passed(10)

In [ ]:
assert(eval(EmptyDict, ex4) == Error)
passed(10)

In [ ]:
assert(eval(EmptyDict, ex5) == NumVal(Positive(two)))
assert(eval(EmptyDict, ex6) == NumVal(Positive(four)))
passed(10)